In [1]:
import sys
sys.path.append("../src/") 
import os

import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import memote
import csv
from cobra import Reaction


In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
#Estimate inf
inf = 1e6
model.solver = 'gurobi'


Set parameter Username
Academic license - for non-commercial use only - expires 2022-06-12


In [3]:

#Define linear relationship between PPFD and Cellular maintainance costs
#This formula comes from Topfer et al (2020) where she defined NGAM in a linear relationship with incident light
def compute_ngam_atp(ppfd):
    v_atp = 0.0049*ppfd + 2.7851
    return v_atp

In [4]:
#Initialize medium so as to induce Photoautotrophic conditions (i.e. no Sucrose import)
model_media = mm.read_medium_csv('../misc/photo_medium.csv', model)
    
model_media['EX_no3(e)'] = inf
model_media['EX_nh4(e)'] = 0
model_media['EX_h2o(e)'] = inf

model_media['EX_h(e)'] = 0
model.medium =model_media

model.medium


{'EX_co2(e)': 1000000,
 'EX_h2o(e)': 1000000.0,
 'EX_no3(e)': 1000000.0,
 'EX_o2(e)': 1000000,
 'EX_pi(e)': 1000000,
 'EX_so4(e)': 1000000,
 'EX_so3(e)': 1000000,
 'EX_h2s(e)': 1000000,
 'EX_fe2(e)': 1000000,
 'EX_fe3(e)': 1000000,
 'EX_mg2(e)': 1000000,
 'EX_photonVis(e)': 1000000}

#Add transgenic reactions to model localized to specific compartment

Transgenic (Line 29 - Ermakova et al)
PEPC = Chloroplastic in M & V (rxn id: PPCc)
PPDK = Chloroplastic in M and BS (rxn id: PPDKs)
NADP-ME = Mitochondrial in M (rxn id: MDHys)
MDH = Chloroplastic in M and BS; Mitochondrial in M  (rxn id: MDH2s, MDH2c)
CA = Cytosolic in M (rxn id: hco3ec)

In WT:
PEPC = Cytosolic in V
PPDK = No signal
NADP-ME = No signal
MDH = Mitochondrial in M 
CA = No signal

Difference:
PEPC = Only in M
PPDK = Chloroplastic M & BS
NADP-ME = Only in M
MDH = Chloroplastic in M/BS
CA = Cytosolic in M



In [ ]:
#My approach would be to add two linear constraints corresponding to the WT and the transgenic
#Enzyme reaction rates, respectively.


In [5]:
# #Add constraints to model
#This code block contains constraints that would simulate the assimilation rates of bs and m cells in a two-cell system (such as those seen near the midvein region of rice leaves)
# #BS photon flux must be the same/less than M flux (Adapted from B&B, 2019)
# photon_import = model.reactions.get_by_id("EX_photonVis(e)")

BS_photon_import = model.reactions.PRISM_white_LED_BS
M_photon_import = model.reactions.PRISM_white_LED_M

#Set photon flux ratio between M/BS cell to be 1:1 or greater 

photon_flux = mm.set_fix_flux_ratio({M_photon_import.id:10, BS_photon_import.id:1},model)

# photon_flux = model.problem.Constraint(M_photon_import.flux_expression 
#                                        - BS_photon_import.flux_expression,
#                                        lb = 0, ub = inf)
model.add_cons_vars(photon_flux)


co2tex_m = model.reactions.CO2tex_M
co2tex_bs = model.reactions.CO2tex_BS

#CO2 Uptake must be constrained to A (Net assimilation rate) which is around 29 +- 1.2 umol CO2 m-2 s-1 for wild type rice
#Lower bound based  on Dark CO2 uptake rates from Ermakova et al (2020)
#CO2 BS intake must be limited owing to its position in the leaf tissue

# # #CO2 intake needs to be mostly in the M cells. Not sure of the exact value:
# # Values from Von Caemmerer (2000) in intermediate C3-C4 photosynthetic plants is 1 mmol CO2 m-2 s-1 per 20 CO2 m-2 s-1, which puts it at 20:1 ratio
# # #try checking Danila et al. (2016) on ratio of surface area of M cell to BS cell
# # #I'll assume that the ratio is at 20:1 in the meantime, meaning that there is minimal gas exchange into the BS cell that would equate to around ~1 umol CO2 m-2s-1
co2_ratio_cons = mm.set_fix_flux_ratio({co2tex_m.id:10, co2tex_bs.id:1},model)
model.add_cons_vars(co2_ratio_cons)

# #Automatic na palang nalalagay siya as constraint

# #First is the BS and M Cell constraints
# #What if I restrict yung import ng O2?

# #Let's try restricting O2 to the value from Lakshmanan et al. (2016) = 3.312 mmol O2 m-2 s-1
# #Actually this value is also reflected in Von Caemmerer's "Modelling C3 metabolism" (pg. 85)
o2tex_bs = model.reactions.O2tex_BS
o2tex_m = model.reactions.O2tex_M

o2_ratio_cons = mm.set_fix_flux_ratio({o2tex_m.id:10, o2tex_bs.id:1},model)
model.add_cons_vars(o2_ratio_cons)

# bs_o2_cons = model.problem.Constraint(0.047*co2tex_bs.flux_expression 
#                                       - 1 * o2tex_bs.flux_expression, lb=0, ub=1000)
# model.add_cons_vars(bs_o2_cons)

In [6]:
# #This code block contains constraints specific for enzyme rate constraints
#This approach is derived from Bogart & Myers (2016) where they constrained the enzyme rate 
#fluxes in each of the 2-cell segments to a specific upper bound while keeping the lower bound
#At 0.


#Maximum values for constraints
wt_pepc = 0 #umol m-2 s-1 #Note: Need to constrain it to 0 pala since no PEPC was detected in either cell type
wt_mdh = 11.18 #umol m-2 s-1
wt_nadp_me = 0.14 #umol m-2 s-1
wt_ppdk = 0.31 #umol m-2 s-1
wt_CA = 7.5 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)

#PEPC constraint (Reaction id: PPCc)
#Need to constrain it to 0 since reaction is only detected in Vascular tissue
pepc_BS = model.reactions.PPCc_BS
pepc_M = model.reactions.PPCc_M

wt_pepc_cons = model.problem.Constraint(pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = wt_pepc)
wt_pepc_cons.name = 'wt_pepc_cons'
model.add_cons_vars(wt_pepc_cons)

#PPDK constraints (Reaction id: PPDKs) (note that this is found in the chloroplast?) 
#Not detected via immunolocalization but enzyme activity is detected

ppdks_BS = model.reactions.PPDKs_BS
ppdks_M = model.reactions.PPDKs_M
wt_ppdks_cons = model.problem.Constraint(ppdks_BS.flux_expression 
                                         + ppdks_M.flux_expression, 
                                         lb = 0, ub = wt_ppdk)
wt_ppdks_cons.name = 'wt_ppdks_cons'
model.add_cons_vars(wt_ppdks_cons)

#Malate Dehydrogenase 
#Only mitochondrial in WT Rice M cells
mdhm_M = model.reactions.MDHm_M
wt_mdh_cons = model.problem.Constraint(mdhm_M.flux_expression,
                                       lb= 0, ub=wt_mdh)
wt_mdh_cons.name = "wt_mdh_cons"
model.add_cons_vars(wt_mdh_cons)


#NADP-ME (Since no signal is detected in WT, no locational constraints are imposed)
#Let's see if I can force it to have a small amount of flux 
nadp_me_M = model.reactions.MDHys_M
nadp_me_BS = model.reactions.MDHys_BS

wt_nadpme_cons = model.problem.Constraint(nadp_me_M.flux_expression
                                         + nadp_me_BS.flux_expression,
                                         lb= 0, ub=wt_nadp_me)
wt_nadpme_cons.name = "wt_nadpme"
model.add_cons_vars(wt_nadpme_cons)

#I should add constraints for Carbonic Anhydrase
#I should constrain it to 0.4 ubar, which would constitute ambient CO2 partial pressure
#HCO3Es
#HCO3Ec
#HCO3Em
#Flux is reversible so constraints are bi-directional


hco3es_m = model.reactions.HCO3Es_M.flux_expression
hco3ec_m = model.reactions.HCO3Ec_M.flux_expression
hco3em_m = model.reactions.HCO3Em_M.flux_expression
hco3es_bs = model.reactions.HCO3Es_BS.flux_expression
hco3ec_bs = model.reactions.HCO3Ec_BS.flux_expression
hco3em_bs = model.reactions.HCO3Em_BS.flux_expression

ca_cons = model.problem.Constraint(hco3es_m + hco3ec_m + hco3em_m 
                                   + hco3es_bs + hco3ec_bs + hco3em_bs,
                                  lb = -wt_CA, ub = wt_CA)
ca_cons.name = 'ca_constraint'
model.add_cons_vars(ca_cons)
#Rbcl constaints

#Let's try to add Rubisco flux ratio

#Retrieve flux expressions oof each RBCl reaction
rbpc_M = model.reactions.RBPCs_M.flux_expression
rbpc_BS = model.reactions.RBPCs_BS.flux_expression
rbpo_M = model.reactions.RBPOs_M.flux_expression
rbpo_BS = model.reactions.RBPOs_BS.flux_expression

#Constraint such that it is limited to 132 umol m-2 s-1
rbcl_vcmax_cons = model.problem.Constraint(rbpc_M + rbpc_BS, lb = 0, ub= 132)

model.add_cons_vars(rbcl_vcmax_cons)

#Constraints for rbcl flux such that v_c/v_o = 3 or higher. 


rbcl_vcvo = model.problem.Constraint(3*(rbpo_M + rbpo_BS) 
                                     - 1*(rbpc_M + rbpc_BS),
                                     lb=0,ub=1000)
rbcl_vcvo.name = 'rbcl_vc/vo_ratio'
model.add_cons_vars(rbcl_vcvo)

#What if I simply constrained that of the M cell one to 3:1?
#This constraint is pretty good actually. 
#This allows the system to be set at a specific Vc/Vo rate while still allowing local variation 
#wherein Rubisco may act in an uncoupled fashion and may have favorable internal vc/vo rates.

print('Successfully added Carbon Fixation constraints')

Successfully added Carbon Fixation constraints


In [7]:
#initialize list of transgenic reactions to add  to model


trans_list = list()


#Transgenic PEPC copy
#PEPC = Chloroplastic in M & V (rxn id: PPCc)
trans_ppcs = Reaction('trans_PPCs_M')
trans_ppcs.name = "Phosphoenolpyruvate carboxylase, plastidic (Transgenic)"

co2_s0 = model.metabolites.co2_s0
h_s0 = model.metabolites.h_s0
pep_s0 = model.metabolites.pep_s0
h2o_s0 = model.metabolites.h2o_s0
oaa_s0 = model.metabolites.oaa_s0
pi_s0 = model.metabolites.pi_s0


#Add metabolites, bounds, and subsystem
trans_ppcs.add_metabolites({co2_s0:-1, h_s0:-1, pep_s0:-1, h2o_s0:1, oaa_s0:1, pi_s0:1})
trans_ppcs.bounds= model.reactions.PPCc_M.bounds
trans_ppcs.subsystem = model.reactions.PPCc_M.subsystem


trans_list.append(trans_ppcs)


#Transgenic PPDK Copy
#Since it already exists I'll just copy and readd it
trans_ppdks_m = Reaction('trans_PPDKs_M')
trans_ppdks_m.add_metabolites(model.reactions.PPDKs_M.metabolites)
trans_ppdks_m.bounds = model.reactions.PPDKs_M.bounds
trans_ppdks_m.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

trans_ppdks_bs = Reaction('trans_PPDKs_BS')
trans_ppdks_bs.add_metabolites(model.reactions.PPDKs_BS.metabolites)
trans_ppdks_bs.bounds = model.reactions.PPDKs_BS.bounds
trans_ppdks_bs.name = "Pyruvate phosphate dikinase, plastidic (Transgenic)"

trans_list.append(trans_ppdks_m)
trans_list.append(trans_ppdks_bs)


#Transgenic NADP-ME
#NADP-ME = Mitochondrial in M
trans_nadp_me = Reaction('trans_MDHym_M')

#retrieve reactants
mal_m0 = model.metabolites.get_by_id('mal-L_m0')
nadp_m0 = model.metabolites.nadp_m0
h_m0 = model.metabolites.h_m0
nadph_m0 = model.metabolites.nadph_m0
oaa_m0 = model.metabolites.oaa_m0

#Add to rxn
trans_nadp_me.add_metabolites({mal_m0:-1, nadp_m0:-1, h_m0:1, nadph_m0:1, oaa_m0:1})
#Add bounds
trans_nadp_me.bounds=(-inf, inf)

trans_list.append(trans_nadp_me)


#Malate Dehydrogenase, mitochondrial (M cell)
trans_MDHm_M = Reaction('trans_MDHm_M')
trans_MDHm_M.name = 'Malate Dehydrogenase, Mitochondrial'
trans_MDHm_M.add_metabolites(model.reactions.MDHm_M.metabolites)
trans_MDHm_M.subsystem = model.reactions.MDHm_M.subsystem

trans_list.append(trans_MDHm_M)

#Malate dehydrogenase, plastidic (M cell)
trans_MDHs_M = Reaction('trans_MDHs_M')
trans_MDHs_M.name = 'Malate Dehydrogenase, Plastidic'
trans_MDHs_M.add_metabolites(model.reactions.MDHs_M.metabolites)
trans_MDHs_M.subsystem = model.reactions.MDHs_M.subsystem

trans_list.append(trans_MDHs_M)

#Malate dehydrogenase, plastidic(BS Cell)
trans_MDHs_BS = Reaction('trans_MDHs_BS')
trans_MDHs_BS.name = 'Malate Dehydrogenase, Plastidic'
trans_MDHs_BS.add_metabolites(model.reactions.MDHs_BS.metabolites)
trans_MDHs_BS.subsystem = model.reactions.MDHs_BS.subsystem

trans_list.append(trans_MDHs_BS)


#Trans CA
#Cytosolic in M
trans_hco3ec_M = Reaction('trans_hco3ec_M')
trans_hco3ec_M.name = 'carbonic anhydrase, cytosolic'
trans_hco3ec_M.add_metabolites(model.reactions.HCO3Ec_M.metabolites)
trans_hco3ec_M.bounds = model.reactions.HCO3Ec_M.bounds

trans_hco3ec_M.subsystem = model.reactions.HCO3Ec_M.subsystem
trans_list.append(trans_hco3ec_M)


#Bulk add to model
model.add_reactions(trans_list)

print("Successfully added transgenic reactions")
print('List of reactions added: ')
for items in trans_list:
    print(items.id)


Successfully added transgenic reactions
List of reactions added: 
trans_PPCs_M
trans_PPDKs_M
trans_PPDKs_BS
trans_MDHym_M
trans_MDHm_M
trans_MDHs_M
trans_MDHs_BS
trans_hco3ec_M


In [8]:
#The following code block is done to
#Define rates
trans_pepc_rates = 7.01 #umol m-2 s-1
trans_ppdks_rates = 3.66 #umol m-2 s-1
trans_mdh_rates = 152.87 #umol m-2 s-1
trans_nadp_me_rates = 0.60 #umol m-2 s-1
trans_CA_rates = 8 #umol m-2  s-1 bar-1 (Constrained to CO2 amounting to 400-500 mbar) (1 bar = 15.74 umol m-2 s-1 CA activity)


#Generate constraints on top of WT constraints

#Retrieve trans reactions
trans_PPCs_M = mm.get_rxn(model, 'trans_PPCs_M')
trans_ppdks_M  = mm.get_rxn(model, 'trans_PPDKs_M')
trans_ppdks_BS = mm.get_rxn(model, 'trans_PPDKs_BS')
trans_MDHm_M = mm.get_rxn(model, 'trans_MDHm_M')
trans_MDHs_M = mm.get_rxn(model, 'trans_MDHs_M')
trans_MDHs_BS = mm.get_rxn(model, 'trans_MDHs_BS')
trans_MDHym_M = mm.get_rxn(model, 'trans_MDHym_M')
trans_hco3ec_M = mm.get_rxn(model, 'trans_hco3ec_M')


#PEPC constraint
trans_pepc_cons = model.problem.Constraint(trans_PPCs_M.flux_expression
                                        +pepc_BS.flux_expression 
                                        + pepc_M.flux_expression, 
                                        lb = 0, ub = trans_pepc_rates)

model.add_cons_vars(trans_pepc_cons)


trans_ppdks_cons = model.problem.Constraint( 
    trans_ppdks_BS.flux_expression +trans_ppdks_M.flux_expression 
    +ppdks_BS.flux_expression + ppdks_M.flux_expression, 
                                         lb = 0, ub = trans_ppdks_rates)
trans_ppdks_cons.name = 'trans_ppdks_cons'
model.add_cons_vars(trans_ppdks_cons)



trans_mdh_cons =  model.problem.Constraint(
   trans_MDHm_M.flux_expression + mdhm_M.flux_expression
    + trans_MDHs_M.flux_expression + trans_MDHs_BS.flux_expression,
                                       lb= 0, ub=trans_mdh_rates)

trans_mdh_cons.name = "trans_mdh_cons"
model.add_cons_vars(trans_mdh_cons)


trans_nadpme_cons = model.problem.Constraint(
    trans_MDHym_M.flux_expression + nadp_me_M.flux_expression + nadp_me_BS.flux_expression,
                                         lb= 0, ub=trans_nadp_me_rates)
trans_nadpme_cons.name = "trans_nadpme"
model.add_cons_vars(trans_nadpme_cons)


trans_ca_cons = model.problem.Constraint(trans_hco3ec_M.flux_expression + hco3es_m + hco3ec_m + hco3em_m 
                                   + hco3es_bs + hco3ec_bs + hco3em_bs,
                                  lb = -trans_CA_rates, ub = trans_CA_rates)
ca_cons.name = 'Trans_CA_cons'
model.add_cons_vars(trans_ca_cons)

print('Successfully added Transgenic-specific constraints')

Successfully added Transgenic-specific constraints


In [9]:
model.repair()

In [10]:
# #This code block is to set a constraint such that M-to-BS cell NGAM ratio is 10-to-1 
# #Similar to what Moreno-Villena (2021) (preprint) had done 

ngam_atp_m = model.reactions.ngam_atp_c_M
ngam_atp_bs = model.reactions.ngam_atp_c_BS
ngam_atp_m.bounds = (0,1000)
ngam_atp_bs.bounds = (0,1000)
ngam_ratio = mm.set_fix_flux_ratio({ngam_atp_m.id:10, ngam_atp_bs.id:1}, model)
ngam_ratio.name = 'ngam_BS/M_ratio'
model.add_cons_vars(ngam_ratio)

print('Successfully added NGAM ratio constraint')

Successfully added NGAM ratio constraint


This code block is to add the constraints for ngam nadphox, which for some reason doesn't save when it was generated using the curation script. I think this is because the constraint objects are only retrieved from the bounds encoded in the SBML.


In [11]:
#Retrieve NGAM reactions
ngam_nadphox_c_M = mm.get_rxn(model, 'ngam_nadphox_c_M')
ngam_nadphox_s_M = mm.get_rxn(model, 'ngam_nadphox_s_M')
ngam_nadphox_m_M = mm.get_rxn(model, 'ngam_nadphox_m_M')
ngam_nadphox_c_BS = mm.get_rxn(model, 'ngam_nadphox_c_BS')
ngam_nadphox_s_BS = mm.get_rxn(model, 'ngam_nadphox_s_BS')
ngam_nadphox_m_BS = mm.get_rxn(model, 'ngam_nadphox_m_BS')


#Set Fixed fluxes
nadphox_c_s_M = mm.set_fix_flux_ratio({ngam_nadphox_c_M.id:1, ngam_nadphox_s_M.id:1},model)
nadphox_c_s_M.name = "nadphox_cs_ratio_M"
nadphox_s_m_M = mm.set_fix_flux_ratio({ngam_nadphox_s_M.id:1, ngam_nadphox_m_M.id:1}, model)
nadphox_s_m_M.name = "nadphox_sm_ratio_M"

nadphox_c_s_BS = mm.set_fix_flux_ratio({ngam_nadphox_c_BS.id:1, ngam_nadphox_s_BS.id:1},model)
nadphox_c_s_BS.name = "nadphox_cs_ratio_BS"
nadphox_s_m_BS = mm.set_fix_flux_ratio({ngam_nadphox_s_BS.id:1, ngam_nadphox_m_BS.id:1}, model)
nadphox_s_m_BS.name = "nadphox_sm_ratio_BS"

#Add constraints
model.add_cons_vars(nadphox_c_s_M)
model.add_cons_vars(nadphox_s_m_M)
model.add_cons_vars(nadphox_c_s_BS)
model.add_cons_vars(nadphox_s_m_BS)

print('successfully added nadphox constraint')

successfully added nadphox constraint


In [12]:

#Retrieve flux expressionns
fex_nadphox_c_M =  mm.get_flux_exp(model, ngam_nadphox_c_M)
fex_nadphox_s_M = mm.get_flux_exp(model, ngam_nadphox_s_M)
fex_nadphox_m_M = mm.get_flux_exp(model, ngam_nadphox_m_M)

fex_nadphox_c_BS =  mm.get_flux_exp(model, ngam_nadphox_c_BS)
fex_nadphox_s_BS =  mm.get_flux_exp(model, ngam_nadphox_s_BS)
fex_nadphox_m_BS =  mm.get_flux_exp(model, ngam_nadphox_m_BS)

fex_atp_c_M = mm.get_flux_exp(model, ngam_atp_m)
fex_atp_c_BS =  mm.get_flux_exp(model, ngam_atp_bs)

#Set the constraint between ATP:NADPH NGAM to 3:1
nadphox_atpase = model.problem.Constraint(3*(fex_nadphox_c_M + fex_nadphox_s_M + fex_nadphox_m_M
                                                   + fex_nadphox_c_BS + fex_nadphox_s_BS + fex_nadphox_m_BS) 
                                     - 1*(fex_atp_c_M + fex_atp_c_BS),
                                     lb=0,ub=0)
nadphox_atpase.name = "nadphox_atpase_ratio"
model.add_cons_vars(nadphox_atpase)

print('successfully added nadphox/atpase ratio')

successfully added nadphox/atpase ratio


In [13]:
file = csv.reader(open('../misc/leaf_inactivated.tsv'), delimiter='\t')
leaf_inactive_rxns = list()
for rows in file:
    row_m = str()
    row_bs = str()
    for rxns in rows:
        row_m += str(rxns) + "_M"
        row_bs += str(rxns) + "_BS"
    leaf_inactive_rxns.append(row_m)
    leaf_inactive_rxns.append(row_bs)

In [14]:

for rxns in model.reactions:
    if rxns.id in leaf_inactive_rxns: 
         print(rxns.id, rxns.bounds)


Coleoptile_Biomass_M (0.0, 1000000.0)
PRISM_blue_LED_M (0.0, 0.0)
PRISM_red_LED_M (0.0, 0.0)
PRISM_green_LED_M (0.0, 0.0)
GDHym_M (-1000000.0, 0.0)
GDHm_M (-1000000.0, 0.0)
AGATx_M (-1000000.0, 0.0)
AGATs_M (-1000000.0, 0.0)
ICDHc_M (-1000000.0, 0.0)
GTHPs_M (0.0, 1000000.0)
GLCNGBc_M (-1000000.0, 0.0)
GLCNGAc_M (-1000000.0, 0.0)
ASPTAm_M (-1000000.0, 0.0)
PDHam1m_M (0.0, 1000000.0)
PDHam2m_M (0.0, 1000000.0)
PDHe2m_M (-1000000.0, 0.0)
PDHe3m_M (-1000000.0, 0.0)
THR3DHx_M (0.0, 1000000.0)
FDHNc_M (0.0, 1000000.0)
SULOm_M (0.0, 1000000.0)
ASNS1c_M (0.0, 1000000.0)
GLYCORc_M (0.0, 1000000.0)
TYRTAs_M (0.0, 1000000.0)
ARGOAT1s_M (-1000000.0, 0.0)
PGI2c_M (-1000000.0, 0.0)
PGI2s_M (-1000000.0, 0.0)
GAMPTc_M (-1000000.0, 0.0)
FTHFLc_M (-1000000.0, 0.0)
CALANc_M (-1000000.0, 0.0)
FORAc_M (-1000000.0, 0.0)
SHMTs_M (-1000000.0, 0.0)
MLTHFRs_M (-1000000.0, 0.0)
METSc_M (-1000000.0, 0.0)
GAPDHys_M (0.0, 1000000.0)
GAPDH2s_M (0.0, 1000000.0)
MTHFDm_M (-1000000.0, 0.0)
MTHFDc_M (-1000000.0, 0.0)
F

In [ ]:
# # This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
# #proper values? I think that would be a better approach, I think.


with model:
                                                                                                                                                                                                                                                                                                                      
    model_media['EX_photonVis(e)'] = 100
    model_media['EX_co2(e)'] = 0
    model_media['EX_o2(e)'] = 0#2.2618
    model.medium = model_media
    
#     #Set NGAM
    ngam_value = compute_ngam_atp(100)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    #Remove thermodynamically infeasible loops
    sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
    
    sample_fluxes_df = sample_fluxes.to_frame()
    
    temp_model = model
    

sample_fluxes
    

In [41]:
# This code block is to test whether a single iteration works. Maybe I should implement the kinetic constraints by Von Caemmerer to obtain proper 
#proper values? I think that would be a better approach, I think.


with model:

    for rxns in model.reactions:
        if rxns.id in leaf_inactive_rxns: #Turn off all reactions in the list loaded above
             rxns.bounds = (0,0)

    model_media['EX_photonVis(e)'] = 1000
    model_media['EX_co2(e)'] = 5
    model_media['EX_o2(e)'] = 2.2#2.2618
    model.medium = model_media
    
#     #Set NGAM
    ngam_value = compute_ngam_atp(200)
    ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                        fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
    model.add_cons_vars(ngam_cons)
#Constrain O2 intake to 3.312 mmol O2 gcw -1 d-1 ~~ 2.2618 umol O2 m-2 s-1 (from Lakshmanan et al. 2016)
    #Change objective function to mature leaf
    model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
    model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
    mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1
    sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
    sample_fluxes_df = sample_fluxes.to_frame()
    
    #Get blocked reactions
    blocked_rxns = cobra.flux_analysis.find_blocked_reactions(model)
    

sample_fluxes
    

/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
Could not get flux for reaction FAC_Osr_BS, setting it to NaN. This is usually due to numerical instability.
/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
Could not get flux for reaction PEDCm_M, setting it to NaN. This is usually due to numerical instability.
/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver status is '{status}'.", UserWarning)
Could not get flux for reaction FAOS183yx_BS, setting it to NaN. This is usually due to numerical instability.
/home/rtsantos3/anaconda3/lib/python3.8/site-packages/cobra/util/solver.py:554: UserWarning: Solver status is 'infeasible'.
  warn(f"Solver sta

Coleoptile_Biomass_M    0.000000
Straw_Biomass_M         0.000000
ACCOAC_OSAc_M           0.000000
ACCOAC_OSAr_M           0.000000
ACPC_OSs_M              0.000000
                          ...   
trans_MDHym_M           0.418344
trans_MDHm_M            1.523168
trans_MDHs_M            0.000000
trans_MDHs_BS           0.000000
trans_hco3ec_M          0.000000
Name: fluxes, Length: 4933, dtype: float64

2520

In [ ]:
#This code  block gives a snapshot of the relevant fluxes on each of the cell types based on the saved sample_fluxes values above
print('rbcl M cell: ', sample_fluxes['RBPCs_M'], 'rbcl BS cell: ',sample_fluxes['RBPCs_BS'])
print('rbcl M cell (photorespiration)', sample_fluxes['RBPOs_M'], 'rbcl BS cell (PR)', sample_fluxes['RBPOs_BS'])
print('vc/vo M:', sample_fluxes['RBPCs_M']/sample_fluxes['RBPOs_M'], 'vc/vo BS:', sample_fluxes['RBPCs_BS']/sample_fluxes['RBPOs_BS'])
print('PEPC M', sample_fluxes['PPCc_M'], 'PEPC BS', sample_fluxes['PPCc_BS'])
print('Carbonic Anhydrase (Cytosolic) M', sample_fluxes['HCO3Ec_M'], 'Carbonic Anhydrase (Cytosolic) BS', sample_fluxes['HCO3Ec_BS'])
print('NADP-ME M', sample_fluxes['MDHys_M'], 'NADP-ME BS', sample_fluxes['MDHys_BS'])
print('Biomass M: ', sample_fluxes['Straw_Biomass_M'], 'Biomass BS', sample_fluxes['Straw_Biomass_BS'])
print('Phloem M: ', sample_fluxes['DM_Phloem_M'], 'Phloem BS', sample_fluxes['DM_Phloem_BS'])
print('co2 consumption M', sample_fluxes[co2tex_m.id], 'co2 consumption BS', sample_fluxes[co2tex_bs.id])
print('o2 consumption M', sample_fluxes['O2tex_M'], 'o2 consumption BS', sample_fluxes['O2tex_BS'])
print('Photosystem II M', sample_fluxes['PSIINC_M'], 'PSII BS', sample_fluxes['PSIINC_BS'])
print('PSI M', sample_fluxes['PSIMR_M'], 'PSI BS', sample_fluxes['PSIMR_BS'])
print('PPFD M: ', sample_fluxes['PRISM_white_LED_M'], 'PPFD BS: ', sample_fluxes['PRISM_white_LED_BS'])

pd_rxn = [x for x in model.reactions if "pd" in x.id and "h2o" not in x.id]
pd_abs_flux = 0
for pds in pd_rxn:
    pd_abs_flux += abs(sample_fluxes[pds.id])
    
print('pd_abs_flux: ', pd_abs_flux)

In [ ]:
# #This code block is to generate flux values thru the network given a set of constraints to PPFD

# ppfd_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))
# ppfd_solns_2cell_fva  = pd.DataFrame(index=list(i.id for i in model.reactions))

# #Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)
   
# for i in range(0,2100,50):
#     with model:
#         ppfd = i 
#         #Set medium to change photon flux and other things and re-add to model
#         model_media['EX_photonVis(e)'] = ppfd
#         model_media['EX_co2(e)'] = 29
#         model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2
#         model.medium = model_media

#         model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 1
#         model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 1



#         #perform pFBA & convert to dataframe
#         tc_soln = cobra.flux_analysis.pfba(model).to_frame()
# #         tc_summary = model.summary(fva=0.95).to_frame()
#         tc_soln = pd.DataFrame({ppfd:tc_soln['fluxes']}, index=list(tc_soln.index))
#         print("PPFD:", ppfd, " RBPC_M cell flux:", tc_soln[ppfd]['RBPCs_M'])
#         #Append to Dataframe
#         ppfd_solns_2cell_fba[ppfd] = tc_soln
#        # ppfd_solns_2cell_fva[ppfd] = 

# #Code now works

In [ ]:
ppfd_solns_2cell_fba = pd.DataFrame(index=list(i.id for i in model.reactions))
ppfd_solns_2cell_fva  = pd.DataFrame(index=list(i.id for i in model.reactions))

#Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)

for i in range(0,2100,50):
    
    #Generate instance of model every iteration
    with model:
        ppfd = i 
        #Set ATP NGAM value to linear PPFD formula

         #Set medium to change photon flux and other things and re-add to model
        model_media['EX_photonVis(e)'] = ppfd
        model_media['EX_co2(e)'] = 29
        model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2 (converted from 3.312 mmol/dcw hr)
        model.medium = model_media

        #Set NGAM value
        ngam_value = compute_ngam_atp(i)
        ngam_cons = model.problem.Constraint(fex_atp_c_M + 
                                            fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
        model.add_cons_vars(ngam_cons)


        #Change objective function to mature leaf
        model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        solution = model.optimize()

        if solution.status == 'optimal':
            sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
            #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
            sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes)
            #Generate dataframe
            sample_fluxes = sample_fluxes.to_frame()

            sample_fluxes = pd.DataFrame({ppfd:sample_fluxes['fluxes']}, index=list(sample_fluxes.index))
            print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes[ppfd]['RBPCs_M'])
            #Append to Dataframe
            ppfd_solns_2cell_fba[ppfd] = sample_fluxes
            
out = open('../flux_results/ppfd_2cell_trans_solns_CCF.tsv', 'w')
ppfd_solns_2cell_fba.to_csv(out, sep='\t')
out.close()
print("Successfully saved PPFD-out (W/ CCF!)")

In [ ]:
# ppfd_solns_2cell_fba_noccf = pd.DataFrame(index=list(i.id for i in model.reactions))
# #Iterate from range 0 to 2000 w/ increments of 50 (max ppfd)

# for i in range(0,2100,50):
    
#     #Generate instance of model every iteration
#     with model:

#         #Set ATP NGAM value to linear PPFD formula

#         #Set medium to change photon flux and other things and re-add to model
#         ppfd = i 
#         model_media['EX_photonVis(e)'] = ppfd
#         model_media['EX_co2(e)'] = 29
#         model_media['EX_o2(e)'] = 2.2618 #umol/m^2 s O2 (converted from 3.312 mmol/dcw hr)
#         model.medium = model_media

#         #Set NGAM value
#         ngam_value = compute_ngam_atp(i)
#         ngam_cons = model.problem.Constraint(fex_atp_c_M + 
#                                             fex_atp_c_BS, lb=ngam_value, ub=ngam_value)
#         model.add_cons_vars(ngam_cons)
        
#         #Change objective function to mature leaf
#         model.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
#         model.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
#         mm.get_rxn(model,'DM_Phloem_BS').objective_coefficient = 1

#         #Optimize then check optimality
#         solution = model.optimize()

#         if solution.status == 'optimal':
#             sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
#             sample_fluxes = sample_fluxes.to_frame()
            
#             #Generate dataframe
#             sample_fluxes = pd.DataFrame({ppfd:sample_fluxes['fluxes']}, index=list(sample_fluxes.index))
#             print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes[ppfd]['RBPCs_M'])
#             #Append to larger Dataframe
#             ppfd_solns_2cell_fba_noccf[ppfd] = sample_fluxes

            
            
# out = open('../flux_results/ppfd_2cell_solns_no_CycleFreeFlux.tsv', 'w')
# ppfd_solns_2cell_fba_noccf.to_csv(out, sep='\t')
# out.close()
# print('Successfully saved PPFD-out (No CCF!)')

In [ ]:
#Constraining O2 consumption to around 3.312 forces water to be consumed by the system, unlike when O2 is unconstrained.
#However, the same behavior when it comes to CO2 assimilation still is being produced.
#Constraining O2 consumption also forces Nitrate consumption to be constrained to realistic levels (when compared to non-constrained O2 where consumption was 10-fold )
#I suspect that this is definitely due to how PPFD flux is structured. 
#A cop-out to this would be to double PPFD per run and measuring PPFD flux in the specific reactions instead.

In [ ]:
#I can probably do the benchmarking right now. The model behaves quite interestingly actually

In [ ]:
#Note: Turning off other prism reactions (red, green, blue, etc) while retaining only white light reactions lowers net carbon assimilation
#I think this is in consideration on how white light is partitioned to several wavelengths, which prevents it from being fully utilized.

In [ ]:
#Other artifacts as of the moment include the following:
#model doesn't produce any Oxygen. Maybe I should add a specific demand reaction for it?
#Based on FVA, the model now produces some oxygen via the demand reaction. 

In [ ]:
#Model also facilitates transfer of metabolites unlike before.

In [ ]:
#Questions:
#Should I restrict Nutrient flow to the Bundle Sheath Cell only or should I allow flux to both the M and BS cell?
#Apparently kasi dito only the Bundle sheath cells produce any biomass
#This is to ensure that only CO2 and Light flux are the only limiting constraints to the system.